In [39]:
# this block imports all necessary libraries and a necessary thing is for this code to work, run ganache 
# and replace the address and publickeys as are required.

from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json


In [40]:
# opening the SCR contract file

with open("./SCR.sol", "r") as file:
    registrationFile = file.read()


In [41]:
#compiling smart contract of certifier(SCR)
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCR.sol": {"content": registrationFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [42]:
# get bytecode
bytecode = compiled_sol["contracts"]["SCR.sol"]["Registration"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(
    compiled_sol["contracts"]["SCR.sol"]["Registration"]["metadata"]
)["output"]["abi"]

In [43]:
# Opening the SCA contract File
    
with open("./SCA.sol", "r") as file:
    AuctioneerFile = file.read()

In [44]:
# compiling SmartContract of auctioneer(SCA)
install_solc("0.8.2")
compiled_solA = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCA.sol": {"content": AuctioneerFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_solA, file)

In [45]:
# get bytecode
bytecodeA = compiled_solA["contracts"]["SCA.sol"]["Submission"]["evm"]["bytecode"]["object"]

# get abi
abiA = json.loads(
    compiled_solA["contracts"]["SCA.sol"]["Submission"]["metadata"]
)["output"]["abi"]

In [46]:
chainID = 1337
chain_id =chainID
httpServer = "HTTP://127.0.0.1:7545"
w3 = Web3(Web3.HTTPProvider(httpServer))

In [47]:


# set up connection for bidder

my_addressB = '0xD10b89b8DaA01289AeB8d4351b5a8a768b2823eF'
private_keyB = '0x0c04edb3ffedaaa3355339c0d39b38bb5a7c96ab76070340610e4739eaa10711'

# set up connection for certifier

my_addressC = '0x2602648B703727b5249Cf0Ff96Ab76afB00301b9'
private_keyC = '0x12f7995fc9b21fea1121bcd8ab153a609b62b9f6f64943bf2b9b40cfa4e1dfff'

# set up connection for auctioneer
my_addressA = '0xa0885dbdaF530bFB3197d22FfC40B793090253fC'
private_keyA = "0xcae39772a5c3b56df7b697d2027828e8bec5534e2cc3724b489879a5433abcb9"

In [48]:
def initalizeBidderContract(etherValue):
    # initialize contract
    nonce = w3.eth.get_transaction_count(my_addressB)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = {
        "chainId": chain_id, 
        "from": my_addressB, 
        "nonce": nonce,
        'to': my_addressA,
        'value' : w3.to_wei(etherValue, 'ether'),
        'gas': 2000000,
        'gasPrice': w3.eth.gas_price
        }
    
    
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyB)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    # print(tx_receipt.gasUsed)

    return [tx_receipt, nonce]


In [49]:
# def bidderPayGas(tx_receipt, nonce,gasAmount):
#     # calling functions in contract block
#     # to work with a contract, you need abi and address
#     #  this line call the the contract by passing the abi and adress of the transactoin

#     fundTransfer_sol = w3.eth.contract(abi=abiB, address=tx_receipt.contractAddress)

#     #  whenever we want to update something we trsact that value so gas is used for 
#     # that purpose we agin perform the transaction

#     call_fun = fundTransfer_sol.functions.sendViaCall(my_addressA,gasAmount).build_transaction(
#         {"chainId": chain_id, "from": my_addressB, "nonce": nonce + 1}
#     )
#     # again sign and same procedure

#     sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyB)
#     tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
#     tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
    

In [50]:
def registrationPhaseFunctiontoInitializeSCR():
    # initialize contract
    Registration = w3.eth.contract(abi=abi, bytecode=bytecode)
    nonce = w3.eth.get_transaction_count(my_addressC)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = Registration.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressC, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyC)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    # print(tx_receipt.gasUsed)

    return [tx_receipt, nonce]

In [51]:
def registrationFunctionForSCRToStoreUID (tx_receipt, nonce,UID):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to update something we trsact that value so gas is used for 
    # that purpose we agin perform the transaction

    call_fun = registration_sol.functions.store(UID).build_transaction(
        {"chainId": chain_id, "from": my_addressC, "nonce": nonce + 1}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyC)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

   


In [52]:
def registrationFunctionForSCRToRetrieveUID(tx_receipt):
    #certifier etrieves the submitted encrypted UID by the contract adress 
    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
    return registration_sol.functions.retrieve().call()


In [53]:
def bidSubmissionPhaseFunctiontoInitializeSCA():
    # initialize contract
    bidSubmission = w3.eth.contract(abi=abiA, bytecode=bytecodeA)
    nonce = w3.eth.get_transaction_count(my_addressA)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = bidSubmission.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyA)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    return [tx_receipt, nonce]

In [54]:
def bidSubmissionFunctionForSCAToStorePB(tx_receipt, nonce,pb,flag):
    
    bid_submission_sol = w3.eth.contract(abi=abiA,address=tx_receipt.contractAddress)                     
    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction
    if flag == 'aucVar':
        call_fun = bid_submission_sol.functions.bidderEncryptedBidSubmission(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1})
    else:#flag ===bidVar
        call_fun = bid_submission_sol.functions.updateBidderVariable(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1})
        
    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyA)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

In [55]:
def bidSubmissionFunctionForSCAToRetrievePB(tx_receipt,flag):
    # calling functions in contract blockand address
    #  this line call the the contract by passing the abi and adress of the transactoin

    bid_submission_sol = w3.eth.contract(abi=abiA, address=tx_receipt.contractAddress)
    
    # just viewing something from your transaction doesnt cost anyting
    if flag == 'aucVar':
        return bid_submission_sol.functions.retrieveBidderEncryptedBidSubmission().call()
    else:
        return bid_submission_sol.functions.fetchVariable().call()


In [56]:
# initalizeBidderContract(100)